# Programação Paralela e Distribuída

Hélio - DC/UFSCar - 2023


# Usando instruções vetoriais do processador

Nos exemplos a seguir, baseados em [1] e [2], ilustra-se o uso de instruções vetoriais do processador. Para tanto, ao invés de programar em Assembly,  usando comandos "inline" usa-se o recurso de operações ***intrisinc*** dentro do próprio programa C. Um guia para as instruções instrinsic pode ser visto em [3].

<br>

[1] https://blog.qiqitori.com/?p=390

[2] https://blog.qiqitori.com/2018/05/matrix-multiplication-using-simd-instructions/

[3] https://software.intel.com/sites/landingpage/IntrinsicsGuide/

    The Intel Intrinsics Guide is an interactive reference tool for Intel intrinsic instructions,
    which are C style functions that provide access to many Intel instructions -
    including Intel® SSE, AVX, AVX-512, and more - without the need to write assembly code.

[4] https://en.wikipedia.org/wiki/Intrinsic_function

```
In computer software, in compiler theory, an intrinsic function (or built-in function) is a function (subroutine)
available for use in a given programming language whose implementation is handled specially by the compiler.
Typically, it may substitute a sequence of automatically generated instructions for the original function call,
similar to an inline function.[1]
Unlike an inline function, the compiler has an intimate knowledge of an intrinsic function
and can thus better integrate and optimize it for a given situation.

Compilers that implement intrinsic functions generally enable them only when a program requests optimization,
otherwise falling back to a default implementation provided by the language runtime system (environment).

Intrinsic functions are often used to explicitly implement vectorization and parallelization in languages
which do not address such constructs.
Some application programming interfaces (API), for example, AltiVec and OpenMP, use intrinsic functions
to declare, respectively, vectorizable and multiprocessing-aware operations during compiling.
The compiler parses the intrinsic functions and converts them into vector math or multiprocessing
object code appropriate for the target platform.
Some intrinsics are used to provide additional constraints to the optimizer, such as values
a variable cannot assume.[2]
```

<br>

**PS**: para quem quiser examinar como é o cógigo assembly gerado por diferentes compiladores e opções de otimização, uma ferramenta interessante é https://godbolt.org/. Com ela, basta digitar o código C, selecionar o compilador e as opções, e examinar o código gerado. Dá para aprender bastante examinando as instruções vetoriais geradas pelas pelo código otimizado!

## Testes com godbolt.org
```
# include <stdio.h>

#define TAM 1024

// #define SUM
// #define REDUCTION
// #define MULT
#define IKJ

#if defined MULT || defined IKJ
 int a[TAM][TAM], b[TAM][TAM], c[TAM][TAM];
# endif

#if defined SUM || defined REDUCTION
 int a[TAM], b[TAM], c[TAM];
#endif

int main()
{
#ifdef SUM
    for(int i=0; i < TAM; i++)
        c[i] = a[i] * b[i];
#endif

#ifdef REDUCTION
    double sum = 0.;
    for (int i=0; i < TAM; i++)
        sum += a[i] * b[i];

    printf("sum: %d\n",sum);
#endif

#ifdef MULT
    for (int i=0; i < TAM; i++)
        for (int j=0; j < TAM; j++)
            for (int k=0; k < TAM; k++)
                c[i][j]+= a[i][k]*b[k][j];
#endif

// Nas otimizações do compilador, experimente -O3 e -mavx !!!
#ifdef IKJ
    for (int i=0; i < TAM; i++)
        for (int k=0; k < TAM; k++)
            for (int j=0; j < TAM; j++)
                c[i][j]+= a[i][k]*b[k][j];
#endif

    return 0;
}
```

# Soma de elementos de um vetor

O programa a seguir usa instruções *intrinsic* para fazer uma operação de soma de 4 valores float em uma única operação.

As opearações realizadas usam registradores internos do processador, com 128 bits, que podem trabalhar com até 4 valores floats (32 bits) de uma só vez.

Para tanto, os valores que serão manipulados devem estar armazenados de forma contígua na memória, como em um vetor com 4 posições, para que dali possam ser copiados para registradores específicos no processador, onde são feitas as operações. Os dados são então copiados de volta para a memória.

Neste caso, serão usadas extensões do repertório de instruções de processadorex x86/64 pertencentes à tecnologia **SSE** , incluindo operações de cópia de valores da memória em registradores ([_mm_loadu_ps](https://software.intel.com/sites/landingpage/IntrinsicsGuide/#text=_mm_loadu&techs=SSE&expand=3407)), soma vetorial ([_mm_add_ps](https://software.intel.com/sites/landingpage/IntrinsicsGuide/#text=_mm_add_ps&expand=3407,133&techs=SSE)), e cópia de valor de registrador para a memória ([_mm_store_ps](https://software.intel.com/sites/landingpage/IntrinsicsGuide/#text=_mm_store_ps&expand=3407,133,5588&techs=SSE,SSE2)).

Nessas operações, o sufixo _ps indica opearação em precisão simples, portanto considerando valores float, de 4 bytes.

Fez sentido?

In [ ]:
%%writefile sse_add.c

// https://blog.qiqitori.com/?p=390

#include <stdio.h>
#include <xmmintrin.h> // Need this in order to be able to use the SSE "intrinsics"
                       // (which provide access to instructions without writing assembly)

int
main(int argc, char **argv)
{
   float a[4], b[4], result[4]; // a and b: input, result: output
   __m128 va, vb, vresult;      // these vars will "point" to SIMD registers


   // initialize arrays (just {0,1,2,3})
   for (int i = 0; i < 4; i++) {
      a[i] = (float)i;
      b[i] = (float)i;
   }

   // load arrays into SIMD registers
   va = _mm_loadu_ps(a);
   vb = _mm_loadu_ps(b);

   // add them together
   vresult = _mm_add_ps(va, vb);

   // store contents of SIMD register into memory
   _mm_storeu_ps(result, vresult);

   // print out result
   for (int i = 0; i < 4; i++) {
      printf("%f\n", result[i]);
   }

   return 0;
}

Writing sse_add.c


In [ ]:
! gcc -Wall sse_add.c -o sse_add && ./sse_add

0.000000
2.000000
4.000000
6.000000


Segue outro programa equivalente, agora usando extensões AVX.

Na compilação deste programa com **gcc**, é preciso incluir o parâmetro ***-mavx***.

https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html




In [ ]:
%%writefile avx_add.c

// https://blog.qiqitori.com/?p=390

#include <immintrin.h>  // Need this in order to be able to use the AVX "intrinsics"
                        //(which provide access to instructions without writing assembly)
#include <stdio.h>

int
main(int argc, char **argv)
{
   // Intel documentation states that we need 32-byte alignment to use _mm256_load_ps/_mm256_store_ps
   float a[8] __attribute__ ((aligned (32)));

   // GCC's syntax makes this look harder than it is:
   // https://gcc.gnu.org/onlinedocs/gcc-6.4.0/gcc/Common-Variable-Attributes.html#Common-Variable-Attributes
   float b[8] __attribute__ ((aligned (32)));

   float result[8]  __attribute__ ((aligned (32)));

   __m256 va, vb, vresult; // __m256 is a 256-bit datatype, so it can hold 8 32-bit floats

   // initialize arrays (just {0,1,2,3,4,5,6,7})
   for (int i = 0; i < 8; i++) {
      a[i] = (float)i;
      b[i] = (float)i;
   }

   // load arrays into SIMD registers
   va = _mm256_load_ps(a); // https://software.intel.com/en-us/node/694474
   vb = _mm256_load_ps(b); // same

   // add them together
   vresult = _mm256_add_ps(va, vb); // https://software.intel.com/en-us/node/523406

   // store contents of SIMD register into memory
   _mm256_store_ps(result, vresult); // https://software.intel.com/en-us/node/694665

   // print out result
   for (int i = 0; i < 8; i++) {
      printf("%f\n", result[i]);
   }

   return 0;
}


Overwriting avx_add.c


In [ ]:
! gcc -Wall avx_add.c -o avx_add -mavx && ./avx_add

0.000000
2.000000
4.000000
6.000000
8.000000
10.000000
12.000000
14.000000


# Multiplicação de matrizes

Mais um exemplo de uso de extensões das instruções do processador (***SSE3***) com chamadas *intrinsics*, desta vez realizando a multiplicação de matrizes.

Nesse caso, como são usadas instruções que realizam operações sobre 4 floats de cada vez, há dois aspectos importantes a considerar:

* primeiro, é preciso que a matriz B esteja no formato transposto para que os dados estejam em posições contíguas da memória
* as matrizes A e B são percorridas em blocos de 4 elementos de cada vez; daí o incremento de k em 4

Outro aspecto importante é como somar os resultados das multiplicações. Afinal, 4 elementos foram multiplicados (2 a 2) em paralelo, e armazenados em 4 posições distintas. Como somar esses 4 valores agora?

Para isso existe uma operação chamada ***hadd*** (*horizontal add*). Essa operação soma os 2 primeiros elementos e armazena o resultado no lugar do 1o. Ao mesmo tempo, soma os elementos 3 e 4 e salva o resultado no lugar do 2o.

Se repetirmos essa operação com o mesmo operador, o resultado é que na posiçãdo do primeiro elemento estará a soma do 1o e o 2o, que continham 1+2 e 3+4.

Pronto!

Vale observar que os dados não são alinhados na memória, daí as operações _u_.

E então, fez sentido?

Ah, na compilação, é preciso prover o parâmetro -msse3, para ativar o uso dessa extensão.

In [ ]:
%%writefile mm_sse_unaligned.c

#include <x86intrin.h>
// #include <xmmintrin.h>      // para _mm_loadu_ps, _mm_mul_ps
// #include <pmmintrin.h>      // para _mm_hadd_ps
// #include <xmmintrin.h>      // para _mm_cvtss_f32
#include <stdio.h>
#include <stdlib.h>

int
main(int argc, char **argv)
{
  float *matrix_a = malloc(1024*1024*sizeof(float));
  float *matrix_b = malloc(1024*1024*sizeof(float));
  float result[1024][1024];
  __m128 va, vb, vresult;

  // initialize matrix_a and matrix_b
  for (int i = 0; i < 1048576; i++) {
    *(matrix_a+i) = 0.1f;
    *(matrix_b+i) = 0.2f;
  }
  // initialize result matrix - poderia substituir por memset()...
  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      result[i][j] = 0;
    }
  }

  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      for (int k = 0; k < 1024; k += 4) {
        // load
        va = _mm_loadu_ps(matrix_a+(i*1024)+k); // matrix_a[i][k]
        vb = _mm_loadu_ps(matrix_b+(j*1024)+k); // matrix_b[j][k]

        // multiply
        vresult = _mm_mul_ps(va, vb);

        // add
        // essa primeira operação soma os elementos (floats) 1 e 2 e os armazena
        // na posição 1, e soma 3 e 4, salvando o resultado na posição 2.
        vresult = _mm_hadd_ps(vresult, vresult);
        // repetindo a operação, teremos a soma de 1 e 2 em 1!
        vresult = _mm_hadd_ps(vresult, vresult);

        // store
        result[i][j] += _mm_cvtss_f32(vresult);
      }
    }
  }

  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      printf("%f ", result[i][j]);
    }
    printf("\n");
  }

  return 0;
}

Writing mm_sse_unaligned.c


In [ ]:
# ! gcc -Wall mm_sse_unaligned.c -o mm_sse_unaligned -O3 -msse3 -fopt-info-optall-optimized -ftree-vectorize
#
# -fopt-info-vec-missed: Detailed info about loops not being vectorized, and a lot of other detailed information.
# ! gcc mm_sse_unaligned.c -o mm_sse_unaligned -msse3 -O3 -fopt-info-vec-missed
#
# -fopt-info-vec-note: Detailed info about all loops and optimizations being done.
# ! gcc mm_sse_unaligned.c -o mm_sse_unaligned -msse3 -O3 -fopt-info-vec-note
#
! gcc -O3 -fopt-info-optall-optimized -msse3 -o mm_sse_unaligned mm_sse_unaligned.c
! time ./mm_sse_unaligned > /dev/null

mm_sse_unaligned.c:56:5: optimized:   Inlining printf/5690 into main/5698 (always_inline).
mm_sse_unaligned.c:54:7: optimized:   Inlining printf/5690 into main/5698 (always_inline).
mm_sse_unaligned.c:47:25: optimized:   Inlining _mm_cvtss_f32/441 into main/5698 (always_inline).
mm_sse_unaligned.c:44:19: optimized:   Inlining _mm_hadd_ps/714 into main/5698 (always_inline).
mm_sse_unaligned.c:42:19: optimized:   Inlining _mm_hadd_ps/714 into main/5698 (always_inline).
mm_sse_unaligned.c:37:19: optimized:   Inlining _mm_mul_ps/337 into main/5698 (always_inline).
mm_sse_unaligned.c:34:14: optimized:   Inlining _mm_loadu_ps/436 into main/5698 (always_inline).
mm_sse_unaligned.c:33:14: optimized:   Inlining _mm_loadu_ps/436 into main/5698 (always_inline).
mm_sse_unaligned.c:23:21: optimized: Loop nest 2 distributed: split to 0 loops and 1 library calls.
mm_sse_unaligned.c:18:21: optimized: loop vectorized using 16 byte vectors

real	0m1.121s
user	0m1.098s
sys	0m0.008s


O próximo exemplo de uso de extensões das instruções do processador realiza a multiplicação de matrizes usando instruções avx256.

Com essas extensões, usamos agora registradores com 256 bits, que são suficientes para armazenar 8 (!) valores float em sequência. Analogamente, a operação de multiplicação ([_mm256_mul_ps](https://software.intel.com/sites/landingpage/IntrinsicsGuide/#expand=3407,133,5588,3333,3333,2946,3928,3407,2946,1890,3931&text=_mm256_mul_ps)) realiza 8 multiplicações em ponto flutuante com precisão simples (float) de uma só vez.

No código, embora a multiplicação seja feita em 8 floats (256 bits) de cada vez, as opearções de soma são feitas em 2 etapas com 128 bits cada. Isso é feito usando máscaras para pegar os valores mais e menos significativos dos 256 bits de cada vez.

Embora exista uma instrução que realize somas de 8 floats de uma só vez ([_mm256_hadd_ps](https://software.intel.com/sites/landingpage/IntrinsicsGuide/#expand=3407,133,5588,3333,3333,3928,3407,2946,1890,3931,2946,2947&text=_mm256_hadd_ps&techs=AVX2)), ela não resolve o problema de agregá-los num valor único.

Aspectos importantes a considerar:

* primeiro, é preciso que a matriz B esteja no formato transposto
* as matrizes A e B são percorridas em blocos de 8 elementos de cada vez; daí o incremento de k em 8

Vale observar que os dados não são alinhados na memória, daí as operações _u_.

Ah, na compilação, é preciso prover o parâmetro ***-mavx***, para ativar o uso dessa extensão.

In [ ]:
%%writefile mm_avx256_unaligned.c

#include <x86intrin.h>
#include <stdio.h>
#include <stdlib.h>

int
main(int argc, char **argv)
{
  float *matrix_a = malloc(1024*1024*sizeof(float));
  float *matrix_b = malloc(1024*1024*sizeof(float));
  float result[1024][1024];
  __m256 va, vb, vtemp;
  __m128 vlow, vhigh, vresult;

  // initialize matrix_a and matrix_b
  for (int i = 0; i < 1048576; i++) {
    *(matrix_a+i) = 0.1f;
    *(matrix_b+i) = 0.2f;
  }
  // initialize result matrix
  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      result[i][j] = 0;
    }
  }

  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      for (int k = 0; k < 1024; k += 8) {
        // load
        va = _mm256_loadu_ps(matrix_a+(i*1024)+k); // matrix_a[i][k]
        vb = _mm256_loadu_ps(matrix_b+(j*1024)+k); // matrix_b[j][k]

        // multiply
        vtemp = _mm256_mul_ps(va, vb);

        // add
        // extract higher four floats
        vhigh = _mm256_extractf128_ps(vtemp, 1); // high 128
        // add higher four floats to lower floats
        vresult = _mm_add_ps(_mm256_castps256_ps128(vtemp), vhigh);
        // horizontal add of that result
        vresult = _mm_hadd_ps(vresult, vresult);
        // another horizontal add of that result
        vresult = _mm_hadd_ps(vresult, vresult);

        // store
        result[i][j] += _mm_cvtss_f32(vresult);
      }
    }
  }

  for (int i = 0; i < 1024; i++) {
    for (int j = 0; j < 1024; j++) {
      printf("%f ", result[i][j]);
    }
    printf("\n");
  }

    return 0;
}

Writing mm_avx256_unaligned.c


In [ ]:
! gcc -O3 -fopt-info-optall-optimized -mavx -o mm_avx256_unaligned mm_avx256_unaligned.c
! time ./mm_avx256_unaligned > /dev/null

mm_avx256_unaligned.c:57:5: optimized:   Inlining printf/5690 into main/5698 (always_inline).
mm_avx256_unaligned.c:55:7: optimized:   Inlining printf/5690 into main/5698 (always_inline).
mm_avx256_unaligned.c:48:25: optimized:   Inlining _mm_cvtss_f32/441 into main/5698 (always_inline).
mm_avx256_unaligned.c:45:19: optimized:   Inlining _mm_hadd_ps/714 into main/5698 (always_inline).
mm_avx256_unaligned.c:43:19: optimized:   Inlining _mm_hadd_ps/714 into main/5698 (always_inline).
mm_avx256_unaligned.c:41:19: optimized:   Inlining _mm_add_ps/335 into main/5698 (always_inline).
mm_avx256_unaligned.c:41:19: optimized:   Inlining _mm256_castps256_ps128/999 into main/5698 (always_inline).
mm_avx256_unaligned.c:39:17: optimized:   Inlining _mm256_extractf128_ps/883 into main/5698 (always_inline).
mm_avx256_unaligned.c:35:17: optimized:   Inlining _mm256_mul_ps/856 into main/5698 (always_inline).
mm_avx256_unaligned.c:32:14: optimized:   Inlining _mm256_loadu_ps/920 into main/5698 (always_i

@h To do: e o ikj?
Gera somas parciais, que podem prover melhor desempenho ainda...